# Projet:  
- Réalisation d'une ApplicationDasboard avec Streamlit  
à partir de base de données open source sur les transactions 
de cartes de crédits.

- Lien web vers la source des données:  
[credits_cards_transactions](https://www.kaggle.com/datasets/priyamchoksi/credit-card-transactions-dataset)

## Importation des librairies

- Installation des librairies:  
Exécuter dans la cellule:  %pip install -r requirements.txt  
ou  
Exécuter dans le terminal (prompt ou powershell):  pip install -r requirements.txt

- Importation des librairies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import random
import warnings 
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",None)
random.seed(123)

## Chargement des données

In [ ]:
# Fonction pour charger les données

def load(file:str, size:int = 10**5, sep:str = ','):
    """--Docstring--
    fonction pour charger les données csv
    tout en optimisant la mémoire du système
    avec les fichiers volumineux. L'option
    size permet de charger les données par
    partition.
    Args:
        file: (string, file.csv or path )
        size: taille de la partition (integer, default = 100000)
              si la mémoire est insuffisante pour importer en un
              bloc.
        sep: séparateur (string, default = <,>)
    """

    try:

        data = pd.read_csv(file, sep=sep, index_col=0)
    
    except MemoryError:

        data = pd.read_csv(file, sep=sep, index_col=0,
                            chunksize = size)
    
    data.index.name = 'index'
    
    return data


In [ ]:
# Chargement des données
path = "credit_card_transactions.csv"
financial_data = load(path) 

## Exploration des données

### affichage des données

In [ ]:
financial_data.head(5)

### description sur les types de variables

In [ ]:
financial_data.columns

In [ ]:
# Fonction pour décrire le dataframe

def info(data):

    Information = pd.DataFrame({
        'Variables': data.columns,
        'Type': data.dtypes,
        'Unique_values': data.nunique(),
        'NA_counts': data.isna().sum(),
        'NA_percent%':data.isna().mean().round(4)*100
        }).reset_index(drop=True)

    return Information

# Application sur financial_data
info(financial_data)

### mise en forme du type des variables

In [ ]:
# fonction pour mettre en forme

def conversion(var):

    # conversion des variables de type = <date>
    date_vars = ('trans_date_trans_time', 'dob')

    if var.name in date_vars:
        
        return pd.to_datetime(var, infer_datetime_format = True)


    # conversion des variables type = <objet>
    if (var.dtype=='object') and (var.name not in date_vars):

        # conversion en string
        if var.nunique() > 20:
            return var.astype('string')

        # conversion en catégorie 
        else:
            return var.astype('category')
    
    else:
        return var 


In [ ]:
# appication de la fonction sur les variables 
financial_data = financial_data.apply(conversion, axis = 0)
info(financial_data)

### Création de nouvelles variables

- Variables dates

In [ ]:
# mettre la variable trans_date_... au format 'jan-01-2020'
# avec une nouvelle variable

financial_data['date_transaction'] = pd.to_datetime(financial_data['trans_date_trans_time']
                                                                  .dt.strftime('%d-%b-%Y'))

# Extraire des informations sur la transaction : jour, mois, heure

financial_data['year_transaction'] = (financial_data['trans_date_trans_time']
                                                   .dt.year)

financial_data['day_transaction'] = (financial_data['trans_date_trans_time']
                                                   .dt.day.astype(int))

financial_data['month_transaction'] = (financial_data['trans_date_trans_time']
                                                   .dt.month_name().astype('category'))

financial_data['hour_transaction'] = (financial_data['trans_date_trans_time']
                                                   .dt.strftime('%H.%M')).astype(float)

# Créer une variable moment de la journée à partir de heure_transaction

condlist = [financial_data['hour_transaction'].between(6,12.59),
            financial_data['hour_transaction'].between(13,18.59),
            financial_data['hour_transaction'].between(19,23.59),
            financial_data['hour_transaction'].between(0,5.59)
            ]
choicelist = ['Morning: 6h-12h59',
              'Afternoon: 13h-18h59',
              'Evening: 19h-23h59',
              'Night: 00h-5h59'
              ]

financial_data['moment_transaction'] = np.select(condlist, choicelist, default=None)
financial_data['moment_transaction'] = financial_data['moment_transaction'].astype('category')

- Variables Âge, Nom, is_fraud

In [ ]:
# Définir la date de référence pour le calcul de l'âge
reference_date = pd.to_datetime('2020-12-31')

# Calcul de l'âge de l'individu à partir de la date de naissance (dob) 
financial_data['age'] = ((reference_date - financial_data['dob']).dt.days/365).astype('int')

# Création de la variable groupe d'âge
financial_data['group_age']=pd.cut(financial_data['age'],
                                   bins=[14,31,46,60,100],
                                   labels=['15-30 ans', '31-45 ans','46-60 ans' ,'+60 ans']
                                  ).astype('category')

# Concaténer first(prénom) et last(nom_de_famille) pour avoir
# nom complet
financial_data['fullname'] = (financial_data['first'] +' '+ financial_data['last']).astype('string')

# Récoder la variable is_fraud 'oui'| 'non' en créant une nouvelle
financial_data['fraud'] = financial_data['is_fraud'].map({1:'oui',0:'non'}).astype('category')


In [ ]:
info(financial_data)

### reorganisation des colonnes et exportation des données finales

In [ ]:
# Réorganisation des columns

financial_data.rename(columns={'category':'business_sector',
                               'cc_num':'card_number',
                               'amt':'amount_USD'}, inplace = True)

list_vars = ['card_number', 'date_transaction','year_transaction' ,'month_transaction',
             'day_transaction', 'hour_transaction', 'moment_transaction','amount_USD',
             'fullname', 'gender', 'age', 'group_age', 'job', 'business_sector',
             'city', 'city_pop', 'state', 'lat', 'long', 'fraud', 'trans_num']

final_data = financial_data[list_vars].sort_values(by = 'date_transaction',
                                      ascending = True,
                                      ignore_index=True)

## Analyse Statistique des données

- fonction pour tableau Croisé de Statistiques descriptives